In [0]:
# Camada Bronze - Card Prices - Magic: The Gathering 
# Objetivo: Processo EL (Extract & Load) com nomenclatura padronizada
# Características: Extract da staging (S3/Parquet) -> Load na bronze (Unity Catalog/Delta) 
# Melhorias: Nomenclatura padronizada, colunas padronizadas, merge incremental robusto

# =============================================================================
# BIBLIOTECAS UTILIZADAS
# =============================================================================
import logging
from datetime import datetime, timedelta
from pyspark.sql.functions import *

# =============================================================================
# CONFIGURAÇÃO DE SEGREDOS
# =============================================================================
def get_secret(secret_name, default_value=None):
    try:
        return dbutils.secrets.get(scope="mtg-pipeline", key=secret_name)
    except:
        if default_value is not None:
            print(f"Segredo '{secret_name}' não encontrado, usando valor padrão")
            return default_value
        else:
            print(f"Segredo obrigatório '{secret_name}' não encontrado")
            raise Exception(f"Segredo '{secret_name}' não configurado")

# =============================================================================
# CONFIGURAÇÕES GLOBAIS
# =============================================================================
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

CATALOG_NAME = get_secret("catalog_name")
SCHEMA_NAME = "bronze"
TABLE_NAME = "TB_BRONZE_CARDPRICES"

S3_BUCKET = get_secret("s3_bucket")
S3_STAGE_PREFIX = get_secret("s3_stage_prefix", "magic_the_gathering/stage")
S3_BRONZE_PREFIX = get_secret("s3_bronze_prefix", "magic_the_gathering/bronze")
S3_STAGE_PATH = f"s3://{S3_BUCKET}/{S3_STAGE_PREFIX}"
S3_BRONZE_PATH = f"{S3_BUCKET}/{S3_BRONZE_PREFIX}"

# =============================================================================
# FUNÇÃO PARA CALCULAR PERÍODO TEMPORAL
# =============================================================================
def calculate_temporal_period(years_back=5):
    # Calcula período de 5 anos completos baseado no ano atual
    current_year = datetime.now().year
    start_year = current_year - years_back
    end_year = current_year  # Incluir o ano atual
    
    cutoff_date = datetime(start_year, 1, 1)
    end_date = datetime(end_year, 12, 31)
    
    return {
        'start_year': start_year,
        'end_year': end_year,
        'cutoff_date': cutoff_date,
        'end_date': end_date,
        'cutoff_date_str': cutoff_date.strftime("%Y-%m-%d"),
        'end_date_str': end_date.strftime("%Y-%m-%d")
    }

# Configurar período temporal
YEARS_BACK = 5
temporal_config = calculate_temporal_period(YEARS_BACK)

START_YEAR = temporal_config['start_year']
END_YEAR = temporal_config['end_year']
CUTOFF_DATE = temporal_config['cutoff_date']
END_DATE = temporal_config['end_date']
CUTOFF_DATE_STR = temporal_config['cutoff_date_str']
END_DATE_STR = temporal_config['end_date_str']

print("Iniciando pipeline EL Unity com nomenclatura padronizada - TB_BRONZE_CARDPRICES")
print(f"Filtro temporal: {YEARS_BACK} anos incluindo ano atual ({START_YEAR} a {END_YEAR})")
print(f"Período: {CUTOFF_DATE_STR} até {END_DATE_STR}")

# =============================================================================
# MAPEAMENTO DE COLUNAS GOV
# =============================================================================
# Mapeamento das colunas originais para nomenclatura GOV (alinhado com Silver)
COLUMN_MAPPING = {
    # Identificação
    'id': 'ID_CARD',
    'name': 'NME_CARD',
    
    # Preços
    'usd': 'VLR_USD',
    'usd_foil': 'VLR_USD_FOIL',
    'eur': 'VLR_EUR',
    'eur_foil': 'VLR_EUR_FOIL',
    'tix': 'VLR_TIX',
    
    # Metadados de preço
    'prices': 'DESC_PRICES',
    'price_updated_at': 'DT_PRICE_UPDATE',
    
    # Informações de mercado
    'market_price': 'VLR_MARKET',
    'market_price_foil': 'VLR_MARKET_FOIL',
    'low_price': 'VLR_LOW',
    'low_price_foil': 'VLR_LOW_FOIL',
    'high_price': 'VLR_HIGH',
    'high_price_foil': 'VLR_HIGH_FOIL',
    
    # Status de preço
    'price_status': 'NME_PRICE_STATUS',
    'price_available': 'FLG_PRICE_AVAILABLE',
    
    # URLs e Links
    'image_url': 'URL_IMAGE',
    'scryfall_uri': 'URL_SCRYFALL',
    
    # Informações de Carta
    'rarity': 'NME_RARITY',
    'set': 'COD_SET',
    
    # Metadados
    'source': 'NME_SOURCE',
    'error': 'DESC_ERROR',
    'ingestion_timestamp': 'DT_INGESTION',
    
    # Colunas de particionamento (se existirem)
    'partition_year': 'RELEASE_YEAR',
    'partition_month': 'RELEASE_MONTH',
    
    # Datas
    'created_at': 'DT_CREATED',
    'updated_at': 'DT_UPDATED'
}



In [0]:
# =============================================================================
# FUNÇÕES UTILITÁRIAS
# =============================================================================
def setup_unity_catalog():
    # Configura o Unity Catalog e schema
    try:
        spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
        spark.sql(f"USE CATALOG {CATALOG_NAME}")
        spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA_NAME}")
        spark.sql(f"USE SCHEMA {SCHEMA_NAME}")
        return True
    except Exception as e:
        print(f"Erro ao configurar Unity Catalog: {e}")
        return False

def extract_from_staging(table_name):
    # EXTRACT: Lê dados da camada staging
    try:
        # Para TB_BRONZE_CARDPRICES, ler do arquivo card_prices da staging
        if table_name == "TB_BRONZE_CARDPRICES":
            stage_path = f"{S3_STAGE_PATH}/*_card_prices.parquet"
        else:
            stage_path = f"{S3_STAGE_PATH}/*_{table_name}.parquet"
        df = spark.read.parquet(stage_path)
        print(f"Extraídos {df.count()} registros de staging para {table_name}")
        return df
    except Exception as e:
        print(f"Erro no EXTRACT de staging: {e}")
        return None

def apply_governance_naming(df):
    # Aplica nomenclatura GOV nas colunas
    try:
        # Obter colunas disponíveis no DataFrame
        available_columns = df.columns
        
        # Criar mapeamento apenas para colunas que existem
        column_mapping = {}
        for original_col, gov_col in COLUMN_MAPPING.items():
            if original_col in available_columns:
                column_mapping[original_col] = gov_col
        
        # Renomear colunas
        for original_col, gov_col in column_mapping.items():
            df = df.withColumnRenamed(original_col, gov_col)
        
        print(f"Aplicada nomenclatura GOV em {len(column_mapping)} colunas")
        print(f"Colunas renomeadas: {list(column_mapping.keys())}")
        
        return df
    except Exception as e:
        print(f"Erro ao aplicar nomenclatura GOV: {e}")
        return df



def apply_temporal_filter(df, table_name):
    # Aplica filtro temporal de 5 anos
    if table_name == "TB_BRONZE_CARDPRICES":
        # Para TB_BRONZE_CARDPRICES, aplicar filtro baseado nos cards existentes
        # Filtro: preços de cards que estão na janela temporal
        from pyspark.sql.functions import col, current_date, date_sub
        
        try:
            # Usar período completo de 5 anos (2020 a 2024)
            from pyspark.sql.functions import to_date
            
            # Ler dados de cards da staging para filtrar preços
            cards_stage_path = f"{S3_STAGE_PATH}/*_cards.parquet"
            print(f"Carregando dados de cards para filtro temporal: {cards_stage_path}")
            
            cards_df = spark.read.parquet(cards_stage_path)
            print(f"Cards carregados: {cards_df.count()} registros")
            
            # Ler dados de sets da staging
            sets_stage_path = f"{S3_STAGE_PATH}/*_sets.parquet"
            print(f"Carregando dados de sets para filtro temporal: {sets_stage_path}")
            
            sets_df = spark.read.parquet(sets_stage_path)
            print(f"Sets carregados: {sets_df.count()} registros")
            
            # Filtrar sets do período completo (5 anos incluindo ano atual)
            sets_filtered = sets_df.filter(
                (col("releaseDate") >= CUTOFF_DATE_STR) & 
                (col("releaseDate") <= END_DATE_STR)
            )
            print(f"Sets filtrados ({START_YEAR} a {END_YEAR}): {sets_filtered.count()} registros")
            
            # Obter lista de códigos de sets válidos
            valid_sets = sets_filtered.select("code").distinct()
            valid_set_codes = [row.code for row in valid_sets.collect()]
            print(f"Sets válidos para filtro: {len(valid_set_codes)} códigos")
            
            # Filtrar cards baseado nos sets válidos
            cards_filtered = cards_df.filter(col("set").isin(valid_set_codes))
            print(f"Cards filtrados por sets válidos: {cards_filtered.count()} registros")
            
            # Obter lista de nomes de cards válidos
            valid_cards = cards_filtered.select("name").distinct()
            valid_card_names = [row.name for row in valid_cards.collect()]
            print(f"Cards válidos para filtro: {len(valid_card_names)} nomes")
            
            # Aplicar filtro nos preços baseado nos cards válidos
            df_filtered = df.filter(col("name").isin(valid_card_names))
            
            # Filtro adicional: garantir que só tenha preços de cards que existem em TB_BRONZE_CARDS
            try:
                bronze_cards_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDS"
                existing_cards_df = spark.table(bronze_cards_table).select("NME_CARD").distinct()
                existing_card_names = [row.NME_CARD for row in existing_cards_df.collect()]
                
                # Aplicar filtro adicional baseado em TB_BRONZE_CARDS
                df_filtered = df_filtered.filter(col("name").isin(existing_card_names))
                
                print(f"Filtro adicional aplicado: {len(existing_card_names)} cards de TB_BRONZE_CARDS")
                
            except Exception as e:
                print(f"Erro ao aplicar filtro adicional baseado em TB_BRONZE_CARDS: {e}")
                print("Continuando apenas com filtro temporal...")
            
            total_before = df.count()
            total_after = df_filtered.count()
            
            print(f"Filtro temporal aplicado para TB_BRONZE_CARDPRICES:")
            print(f"  - Período: {CUTOFF_DATE_STR} até {END_DATE_STR}")
            print(f"  - Anos incluindo atual: {START_YEAR} a {END_YEAR}")
            print(f"  - Sets válidos: {len(valid_set_codes)}")
            print(f"  - Cards válidos: {len(valid_card_names)}")
            print(f"  - Registros antes: {total_before}")
            print(f"  - Registros depois: {total_after}")
            print(f"  - Registros removidos: {total_before - total_after}")
            
            return df_filtered
            
        except Exception as e:
            print(f"Erro ao aplicar filtro temporal baseado em cards: {e}")
            print("Continuando sem filtro temporal...")
            return df
    
    return df

def check_delta_table_exists(delta_path):
    # Verifica se a tabela Delta existe
    try:
        # Verificar se _delta_log existe
        delta_log_path = f"{delta_path}/_delta_log"
        try:
            files = dbutils.fs.ls(delta_log_path)
            if files:
                # Tentar ler a tabela para verificar se tem dados
                df = spark.read.format("delta").load(delta_path)
                count = df.count()
                print(f"Tabela Delta existe com {count} registros")
                return True, count > 0
            else:
                return False, False
        except:
            return False, False
    except Exception as e:
        print(f"Erro ao verificar tabela Delta: {e}")
        return False, False

def get_delta_schema_for_merge(delta_path):
    # Obtém o schema da tabela Delta existente, excluindo colunas de particionamento
    try:
        df = spark.read.format("delta").load(delta_path)
        schema_fields = [field.name for field in df.schema.fields]
        
        # Excluir colunas de particionamento do schema para merge
        partition_columns = ['RELEASE_YEAR', 'RELEASE_MONTH']
        merge_schema = [col for col in schema_fields if col not in partition_columns]
        
        print(f"Schema completo da tabela Delta: {schema_fields}")
        print(f"Schema para merge (sem particionamento): {merge_schema}")
        return merge_schema
    except Exception as e:
        print(f"Erro ao obter schema Delta: {e}")
        return []

def prepare_dataframe_for_merge(df, table_name, target_schema=None):
    # Prepara DataFrame para merge, garantindo compatibilidade de schema
    if not df:
        return None
    
    try:
        # Aplicar nomenclatura GOV
        df = apply_governance_naming(df)
        
        # Elimina duplicidade de NME_CARD no DataFrame de origem
        total_before = df.count()
        df = df.dropDuplicates(['NME_CARD'])
        total_after = df.count()
        print(f"Removidas {total_before - total_after} duplicatas baseadas em 'NME_CARD'")
        
        # Aplicar filtro temporal
        df = apply_temporal_filter(df, table_name)
        
        # Se temos schema de destino, filtrar colunas compatíveis
        if target_schema:
            available_columns = df.columns
            compatible_columns = [col for col in available_columns if col in target_schema]
            
            if not compatible_columns:
                print("Nenhuma coluna compatível encontrada para merge")
                return None
            
            print(f"Colunas compatíveis para merge: {compatible_columns}")
            df = df.select(compatible_columns)
        
        return df
    except Exception as e:
        print(f"Erro ao preparar DataFrame para merge: {e}")
        return None

def execute_merge_with_specific_columns(delta_table, merge_df, merge_columns):
    # Executa merge especificando exatamente quais colunas atualizar
    try:
        # Construir a condição de merge (card_prices usa 'NME_CARD' como chave)
        merge_condition = "bronze.NME_CARD = novo.NME_CARD"
        
        # Construir as ações de UPDATE especificando colunas
        update_actions = {}
        for col_name in merge_columns:
            if col_name != 'NME_CARD':  # Não atualizar a chave de merge
                update_actions[col_name] = f"novo.{col_name}"
        
        # Construir as ações de INSERT especificando colunas
        insert_actions = {}
        for col_name in merge_columns:
            insert_actions[col_name] = f"novo.{col_name}"
        
        print(f"Executando merge com {len(update_actions)} colunas para UPDATE")
        print(f"Colunas para UPDATE: {list(update_actions.keys())}")
        
        # Executar merge com ações específicas
        delta_table.alias("bronze").merge(
            merge_df.alias("novo"),
            merge_condition
        ).whenMatchedUpdate(set=update_actions) \
         .whenNotMatchedInsert(values=insert_actions) \
         .execute()
        
        print("Merge Delta executado com sucesso")
        return True
    except Exception as e:
        print(f"Erro no merge Delta: {e}")
        return False

def check_unity_table_exists(full_table_name):
    # Verifica se a tabela Unity Catalog existe
    try:
        # Tentar fazer uma consulta simples na tabela
        test_query = f"SELECT 1 FROM {full_table_name} LIMIT 1"
        spark.sql(test_query)
        print(f"Tabela Unity Catalog '{full_table_name}' existe")
        return True
    except Exception as e:
        print(f"Tabela Unity Catalog '{full_table_name}' não existe ou não está acessível")
        return False

def create_or_update_unity_catalog(full_table_name, delta_path):
    # Cria ou atualiza tabela Unity Catalog preservando modificações existentes
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            # Criar tabela pela primeira vez
            print(f"Criando tabela Unity Catalog: {full_table_name}")
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS {full_table_name}
            USING DELTA
            LOCATION '{delta_path}'
            COMMENT 'Tabela bronze de fato de preços de cards do Magic: The Gathering com nomenclatura padronizada'
            """
            spark.sql(create_table_sql)
            print(f"Tabela Unity Catalog criada: {full_table_name}")
        else:
            # Tabela já existe, apenas atualizar propriedades de pipeline (sem perder modificações)
            print(f"Tabela Unity Catalog já existe: {full_table_name}")
            print("Atualizando apenas propriedades de pipeline (preservando modificações existentes)")
        
        # Sempre atualizar propriedades de pipeline (não afeta modificações customizadas)
        try:
            spark.sql(f"""
            ALTER TABLE {full_table_name} SET TBLPROPERTIES (
                'bronze_layer' = 'true',
                'data_source' = 'scryfall_api',
                'last_processing_date' = '{datetime.now().strftime("%Y-%m-%d %H:%M:%S")}',
                'table_type' = 'bronze',
                'load_mode' = 'incremental_merge_gov',
                'governance_applied' = 'true',
                'naming_convention' = 'GOV',
                'temporal_window_years' = '{YEARS_BACK}',
                'temporal_filter_source' = 'cards_sets_releaseDate',
                'temporal_filter_column' = 'name',
                'partitioning' = 'RELEASE_YEAR_MONTH'
            )
            """)
            print("Propriedades de pipeline atualizadas")
        except Exception as e:
            print(f"Aviso: Não foi possível atualizar propriedades de pipeline: {e}")
            
    except Exception as e:
        print(f"Erro ao criar/atualizar tabela Unity Catalog: {e}")

def load_to_bronze_unity_incremental_gov(df, table_name):
    # LOAD: Carrega dados na camada bronze (Unity + Incremental + GOV)
    if not df:
        return None
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        from delta.tables import DeltaTable
        
        # Verificar se tabela Delta existe
        delta_exists, has_data = check_delta_table_exists(delta_path)
        
        if not delta_exists or not has_data:
            # Primeira criação com particionamento
            print("Criando tabela Delta pela primeira vez com nomenclatura GOV...")
            df_processed = prepare_dataframe_for_merge(df, table_name)
            
            if table_name == "TB_BRONZE_CARDPRICES":
                # Para TB_BRONZE_CARDPRICES, particionamento por ano de lançamento do set
                # JOIN com cards e sets para pegar releaseDate real
                from pyspark.sql.functions import current_timestamp, lit, year, month
                
                # Carregar dados de cards
                cards_stage_path = f"{S3_STAGE_PATH}/*_cards.parquet"
                print(f"Carregando cards para particionamento: {cards_stage_path}")
                cards_df = spark.read.parquet(cards_stage_path)
                
                # Carregar dados de sets
                sets_stage_path = f"{S3_STAGE_PATH}/*_sets.parquet"
                print(f"Carregando sets para particionamento: {sets_stage_path}")
                sets_df = spark.read.parquet(sets_stage_path)
                
                # Selecionar apenas colunas necessárias
                cards_for_join = cards_df.select("name", "set")
                sets_for_join = sets_df.select("code", "releaseDate")
                print(f"Cards carregados para JOIN: {cards_for_join.count()} registros")
                print(f"Sets carregados para JOIN: {sets_for_join.count()} registros")
                
                # JOIN com cards para pegar set (renomeando 'set' para 'card_set')
                df_with_cards = df_processed.join(
                    cards_for_join.withColumnRenamed("set", "card_set"),
                    df_processed.NME_CARD == cards_for_join.name,
                    "left"
                )
                
                # JOIN com sets para pegar releaseDate
                df_with_sets = df_with_cards.join(
                    sets_for_join,
                    df_with_cards.card_set == sets_for_join.code,
                    "left"
                )
                
                # Criar colunas de particionamento baseadas no releaseDate real
                df_with_partition = df_with_sets.withColumn("RELEASE_YEAR", 
                                  year(col("releaseDate"))) \
                       .withColumn("RELEASE_MONTH", 
                                  month(col("releaseDate")))
                
                # Remover colunas de JOIN desnecessárias
                df_with_partition = df_with_partition.drop("name", "card_set", "code", "releaseDate")
                
                print(f"Particionamento criado com releaseDate real dos sets")
                
                df_with_partition.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
                       .save(delta_path)
            else:
                df_processed.write.format("delta") \
                       .mode("overwrite") \
                       .option("overwriteSchema", "true") \
                       .save(delta_path)
            print("Tabela Delta criada com sucesso")
        else:
            # Para merge, obter schema da tabela existente (excluindo particionamento) e preparar dados
            print("Tabela Delta existe, preparando para merge...")
            target_schema = get_delta_schema_for_merge(delta_path)
            
            # Preparar DataFrame para merge
            merge_df = prepare_dataframe_for_merge(df, table_name, target_schema)
            if not merge_df:
                print("Nenhum DataFrame compatível para merge")
                return None
            
            # Executar merge com colunas específicas (sem particionamento)
            print("Executando merge Delta...")
            delta_table = DeltaTable.forPath(spark, delta_path)
            
            # Usar função específica para merge com colunas definidas
            merge_success = execute_merge_with_specific_columns(delta_table, merge_df, target_schema)
            if not merge_success:
                print("Falha no merge Delta")
                return None

        # Criar ou atualizar tabela Unity Catalog (PRESERVANDO MODIFICAÇÕES)
        create_or_update_unity_catalog(full_table_name, delta_path)
        
        return df
    except Exception as e:
        print(f"Erro no LOAD para bronze {table_name}: {e}")
        return None

def process_el_unity_incremental_gov(table_name):
    # Executa o pipeline EL Unity + Incremental + GOV
    stage_df = extract_from_staging(table_name)
    if not stage_df:
        print(f"Falha no EXTRACT de staging para {table_name}")
        return None
    
    result_df = load_to_bronze_unity_incremental_gov(stage_df, table_name)
    return result_df

def query_bronze_unity(table_name):
    # Consulta na tabela bronze Unity Catalog
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        count_query = f"SELECT COUNT(*) as total FROM {full_table_name}"
        count_result = spark.sql(count_query)
        count_result.show()
        
    except Exception as e:
        print(f"Erro ao consultar tabela bronze: {e}")

def show_delta_info(table_name):
    # Mostra informações da tabela Delta
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        from delta.tables import DeltaTable
        delta_table = DeltaTable.forPath(spark, delta_path)
        history = delta_table.history()
        print(f"Versões Delta: {history.count()}")
            
    except Exception as e:
        print(f"Erro ao mostrar informações Delta: {e}")

def show_sample_data(table_name):
    # Mostra dados de exemplo da tabela
    try:
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        sample_query = f"SELECT * FROM {full_table_name} LIMIT 5"
        print("Dados de exemplo:")
        spark.sql(sample_query).show(truncate=False)
        
    except Exception as e:
        print(f"Erro ao mostrar dados de exemplo: {e}")

def check_partitioning_working(table_name):
    # Verifica se o particionamento está funcionando
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        
        # Verificar se existem partições
        try:
            partitions = dbutils.fs.ls(delta_path)
            has_partitions = any("RELEASE_YEAR=" in p.path for p in partitions)
            
            if has_partitions:
                print("Particionamento funcionando: Encontradas partições RELEASE_YEAR")
                return True
            else:
                print("Particionamento não encontrado: Nenhuma partição RELEASE_YEAR")
                return False
                
        except Exception as e:
            print(f"Erro ao verificar particionamento: {e}")
            return False
            
    except Exception as e:
        print(f"Erro ao verificar particionamento: {e}")
        return False

def show_governance_info():
    # Mostra informações sobre a governança aplicada
    print("=" * 60)
    print("INFORMAÇÕES DE GOVERNAÇA - TB_BRONZE_CARDPRICES")
    print("=" * 60)
    print(f"Nomenclatura GOV aplicada: {len(COLUMN_MAPPING)} colunas")
    print(f"Período temporal: {YEARS_BACK} anos incluindo atual ({START_YEAR} a {END_YEAR})")
    print(f"Filtro baseado em: Cards e Sets com releaseDate")
    print(f"Particionamento: RELEASE_YEAR e RELEASE_MONTH")
    print(f"Chave de merge: NME_CARD")
    print("=" * 60)

def check_unity_catalog_needs_update(full_table_name, delta_path):
    # Verifica se o Unity Catalog precisa ser atualizado
    try:
        table_exists = check_unity_table_exists(full_table_name)
        
        if not table_exists:
            print(f"Tabela Unity Catalog não existe: {full_table_name}")
            return True
        
        # Verificar se a tabela está apontando para o local correto
        try:
            # Tentar fazer uma consulta simples
            test_query = f"SELECT 1 FROM {full_table_name} LIMIT 1"
            spark.sql(test_query)
            print(f"Tabela Unity Catalog OK: {full_table_name}")
            return False
        except Exception as e:
            print(f"Tabela Unity Catalog com problema: {e}")
            return True
            
    except Exception as e:
        print(f"Erro ao verificar Unity Catalog: {e}")
        return True

def clean_cardprices_consistency():
    # Remove preços de cards que não existem em TB_BRONZE_CARDS
    try:
        print("=" * 60)
        print("LIMPEZA DE CONSISTÊNCIA - TB_BRONZE_CARDPRICES")
        print("=" * 60)
        
        bronze_cards_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDS"
        bronze_prices_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDPRICES"
        
        # Obter cards válidos de TB_BRONZE_CARDS
        valid_cards_df = spark.table(bronze_cards_table).select("NME_CARD").distinct()
        valid_card_names = [row.NME_CARD for row in valid_cards_df.collect()]
        print(f"Cards válidos em TB_BRONZE_CARDS: {len(valid_card_names)}")
        
        # Obter todos os preços
        all_prices_df = spark.table(bronze_prices_table)
        total_prices_before = all_prices_df.count()
        print(f"Total de preços antes da limpeza: {total_prices_before}")
        
        # Filtrar apenas preços de cards válidos
        filtered_prices_df = all_prices_df.filter(col("NME_CARD").isin(valid_card_names))
        total_prices_after = filtered_prices_df.count()
        print(f"Total de preços após filtro: {total_prices_after}")
        print(f"Preços removidos: {total_prices_before - total_prices_after}")
        
        # Recriar tabela apenas com preços válidos
        delta_path = f"s3://{S3_BRONZE_PATH}/TB_BRONZE_CARDPRICES"
        
        # Salvar dados filtrados
        filtered_prices_df.write.format("delta") \
               .mode("overwrite") \
               .option("overwriteSchema", "true") \
               .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
               .save(delta_path)
        
        print("Tabela TB_BRONZE_CARDPRICES recriada apenas com preços válidos")
        print("=" * 60)
        
        return True
        
    except Exception as e:
        print(f"Erro na limpeza de consistência: {e}")
        return False

def force_recreate_table_with_correct_schema(table_name):
    # Força a recriação da tabela com schema correto
    try:
        delta_path = f"s3://{S3_BRONZE_PATH}/{table_name}"
        full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.{table_name}"
        
        print(f"Forçando recriação da tabela {table_name} com schema correto...")
        
        # Remover tabela Delta existente
        try:
            dbutils.fs.rm(delta_path, recurse=True)
            print(f"Tabela Delta removida: {delta_path}")
        except Exception as e:
            print(f"Erro ao remover tabela Delta: {e}")
        
        # Remover tabela Unity Catalog se existir
        try:
            spark.sql(f"DROP TABLE IF EXISTS {full_table_name}")
            print(f"Tabela Unity Catalog removida: {full_table_name}")
        except Exception as e:
            print(f"Erro ao remover tabela Unity Catalog: {e}")
        
        # Recriar tabela com schema correto
        print("Recriando tabela com schema correto...")
        
        # Extrair dados da staging
        stage_df = extract_from_staging(table_name)
        if not stage_df:
            print("Falha ao extrair dados da staging")
            return False
        
        # Aplicar nomenclatura GOV
        stage_df = apply_governance_naming(stage_df)
        
        # Aplicar filtro temporal
        stage_df = apply_temporal_filter(stage_df, table_name)
        
        # Criar tabela com particionamento correto
        if table_name == "TB_BRONZE_CARDPRICES":
            # Para TB_BRONZE_CARDPRICES, particionamento por ano de lançamento do set
            from pyspark.sql.functions import current_timestamp, lit, year, month
            
            # Carregar dados de cards
            cards_stage_path = f"{S3_STAGE_PATH}/*_cards.parquet"
            print(f"Carregando cards para particionamento: {cards_stage_path}")
            cards_df = spark.read.parquet(cards_stage_path)
            
            # Carregar dados de sets
            sets_stage_path = f"{S3_STAGE_PATH}/*_sets.parquet"
            print(f"Carregando sets para particionamento: {sets_stage_path}")
            sets_df = spark.read.parquet(sets_stage_path)
            
            # Selecionar apenas colunas necessárias
            cards_for_join = cards_df.select("name", "set")
            sets_for_join = sets_df.select("code", "releaseDate")
            
            # JOIN com cards para pegar set (renomeando 'set' para 'card_set')
            df_with_cards = stage_df.join(
                cards_for_join.withColumnRenamed("set", "card_set"),
                stage_df.NME_CARD == cards_for_join.name,
                "left"
            )
            
            # JOIN com sets para pegar releaseDate
            df_with_sets = df_with_cards.join(
                sets_for_join,
                df_with_cards.card_set == sets_for_join.code,
                "left"
            )
            
            # Criar colunas de particionamento baseadas no releaseDate real
            df_with_partition = df_with_sets.withColumn("RELEASE_YEAR", 
                              year(col("releaseDate"))) \
                   .withColumn("RELEASE_MONTH", 
                              month(col("releaseDate")))
            
            # Remover colunas de JOIN desnecessárias
            df_with_partition = df_with_partition.drop("name", "card_set", "code", "releaseDate")
            
            print(f"Particionamento criado com releaseDate real dos sets")
            
            # Criar tabela Delta
            df_with_partition.write.format("delta") \
                   .mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .partitionBy("RELEASE_YEAR", "RELEASE_MONTH") \
                   .save(delta_path)
        
        # Criar tabela Unity Catalog
        create_or_update_unity_catalog(full_table_name, delta_path)
        
        print(f"Tabela {table_name} recriada com schema correto")
        
        # Verificar schema após recriação
        print("\nVerificando schema após recriação...")
        try:
            df = spark.read.format("delta").load(delta_path)
            schema_fields = [field.name for field in df.schema.fields]
            print(f"Schema da tabela recriada: {schema_fields}")
            
            # Verificar se todas as colunas estão com nomenclatura correta
            expected_columns = [
                'VLR_EUR', 'URL_IMAGE', 'DT_INGESTION', 'NME_CARD', 'NME_RARITY',
                'URL_SCRYFALL', 'COD_SET', 'NME_SOURCE', 'VLR_TIX', 'VLR_USD',
                'DESC_ERROR', 'RELEASE_YEAR', 'RELEASE_MONTH'
            ]
            
            missing_columns = [col for col in expected_columns if col not in schema_fields]
            incorrect_columns = [col for col in schema_fields if col in [
                'image_url', 'rarity', 'scryfall_uri', 'set', 'source', 'error',
                'partition_year', 'partition_month'
            ]]
            
            if missing_columns:
                print(f"Colunas faltando: {missing_columns}")
            if incorrect_columns:
                print(f"Colunas com nomenclatura incorreta: {incorrect_columns}")
            if not missing_columns and not incorrect_columns:
                print("✅ Schema correto - Todas as colunas estão com nomenclatura GOV")
            
        except Exception as e:
            print(f"Erro ao verificar schema: {e}")
        
        return True
        
    except Exception as e:
        print(f"Erro ao recriar tabela: {e}")
        return False



In [0]:
# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

try:
    spark
except NameError:
    from pyspark.sql import SparkSession
    spark = SparkSession.builder.getOrCreate()

setup_success = setup_unity_catalog()
if not setup_success:
    raise Exception("Falha ao configurar Unity Catalog")

# Executar pipeline
card_prices_bronze_df = process_el_unity_incremental_gov("TB_BRONZE_CARDPRICES")

# Verificar se precisa recriar tabela (apenas se schema estiver incorreto)
print("\n" + "=" * 60)
print("VERIFICAÇÃO DE SCHEMA")
print("=" * 60)

delta_path = f"s3://{S3_BRONZE_PATH}/TB_BRONZE_CARDPRICES"
try:
    df = spark.read.format("delta").load(delta_path)
    schema_fields = [field.name for field in df.schema.fields]
    
    # Verificar se há colunas com nomenclatura incorreta
    incorrect_columns = [col for col in schema_fields if col in [
        'image_url', 'rarity', 'scryfall_uri', 'set', 'source', 'error',
        'partition_year', 'partition_month'
    ]]
    
    if incorrect_columns:
        print(f"Schema incorreto detectado: {incorrect_columns}")
        print("Recriando tabela com schema correto...")
        force_recreate_table_with_correct_schema("TB_BRONZE_CARDPRICES")
    else:
        print("✅ Schema correto - Tabela não precisa ser recriada")
        
except Exception as e:
    print(f"Erro ao verificar schema: {e}")
    print("Recriando tabela por segurança...")
    force_recreate_table_with_correct_schema("TB_BRONZE_CARDPRICES")

# Verificar consistência (apenas se necessário)
print("\n" + "=" * 60)
print("VERIFICAÇÃO DE CONSISTÊNCIA")
print("=" * 60)

try:
    bronze_cards_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDS"
    bronze_prices_table = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDPRICES"
    
    # Verificar se há inconsistência
    df_cards = spark.table(bronze_cards_table).select("NME_CARD").distinct()
    df_prices = spark.table(bronze_prices_table).select("NME_CARD").distinct()
    
    prices_not_in_cards = df_prices.join(df_cards, "NME_CARD", "left_anti")
    inconsistent_count = prices_not_in_cards.count()
    
    if inconsistent_count > 0:
        print(f"Inconsistência detectada: {inconsistent_count} cards extras em TB_BRONZE_CARDPRICES")
        print("Executando limpeza de consistência...")
        clean_cardprices_consistency()
    else:
        print("✅ Consistência OK - Não é necessário limpeza")
        
except Exception as e:
    print(f"Erro ao verificar consistência: {e}")
    print("Executando limpeza por segurança...")
    clean_cardprices_consistency()

# Verificar Unity Catalog (apenas se necessário)
print("\n" + "=" * 60)
print("VERIFICAÇÃO DO UNITY CATALOG")
print("=" * 60)

full_table_name = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDPRICES"
delta_path = f"s3://{S3_BRONZE_PATH}/TB_BRONZE_CARDPRICES"

if check_unity_catalog_needs_update(full_table_name, delta_path):
    print("Atualizando Unity Catalog...")
    create_or_update_unity_catalog(full_table_name, delta_path)
else:
    print("✅ Unity Catalog OK - Não precisa ser atualizado")

if card_prices_bronze_df:
    print("Pipeline executado com sucesso")
    
    # Verificações e informações
    query_bronze_unity("TB_BRONZE_CARDPRICES")
    show_delta_info("TB_BRONZE_CARDPRICES")
    #show_sample_data("TB_BRONZE_CARDPRICES")
    check_partitioning_working("TB_BRONZE_CARDPRICES")
    show_governance_info()
else:
    print("Falha no pipeline") 

# =============================================================================
# VALIDAÇÃO DE CONSISTÊNCIA ENTRE CARDS E CARDPRICES
# =============================================================================
try:
    bronze_cards = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDS"
    bronze_prices = f"{CATALOG_NAME}.{SCHEMA_NAME}.TB_BRONZE_CARDPRICES"

    df_cards = spark.table(bronze_cards).select("NME_CARD").distinct()
    df_prices = spark.table(bronze_prices).select("NME_CARD").distinct()

    # Cartas presentes em cards mas não em prices
    cards_not_in_prices = df_cards.join(df_prices, "NME_CARD", "left_anti")
    print(f"Cartas presentes em TB_BRONZE_CARDS mas não em TB_BRONZE_CARDPRICES: {cards_not_in_prices.count()}")
    if cards_not_in_prices.count() > 0:
        cards_not_in_prices.show(truncate=False)

    # Cartas presentes em prices mas não em cards
    prices_not_in_cards = df_prices.join(df_cards, "NME_CARD", "left_anti")
    print(f"Cartas presentes em TB_BRONZE_CARDPRICES mas não em TB_BRONZE_CARDS: {prices_not_in_cards.count()}")
    if prices_not_in_cards.count() > 0:
        prices_not_in_cards.show(truncate=False)

    # Contagem total
    print(f"Total de cartas em TB_BRONZE_CARDS: {df_cards.count()}")
    print(f"Total de cartas em TB_BRONZE_CARDPRICES: {df_prices.count()}")

    if cards_not_in_prices.count() == 0 and prices_not_in_cards.count() == 0:
        print("Validação de consistência OK: As duas tabelas possuem as mesmas cartas.")
    else:
        print("Inconsistência detectada entre as tabelas de cards e cardprices!")
except Exception as e:
    print(f"Erro na validação de consistência entre cards e cardprices: {e}") 